<a href="https://colab.research.google.com/github/Ellen-Tuane/camera_calibration/blob/main/camera_calibration_and_undistortions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Camera Calibration and Distortion

1 . Enable GPU on notbool settings: Edit > Notebook Settings > on hardware aceleration chosse GPU and save it.

2 . Amount Google Drive to import **images**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

3 . Copy images from drive into images folder on colab

In [ ]:
!cp -a /content/drive/MyDrive/yolov4/Calibration/.   /content/images/

4. Move into images folder

In [ ]:
%cd /content/images

5. Set directory where images output will be seved

In [ ]:
save_path = '/content/drive/MyDrive/yolov4/calibration_output '

6 . Import Labraries

In [ ]:
# Import required modules
import cv2
import numpy as np
import os
import glob

7 . Set chessboard dimentions

In [ ]:
# Define the dimensions of checkerboard
CHECKERBOARD = (9, 6)

8 . Set image type

In [ ]:
# Extracting path of individual image stored in a given directory. Since no path is
# specified, it will take current directory images files alone

images = glob.glob('*.HEIC') # set image type (.png, .jpg, .jpeg, .HEIC, ...)

print(len(images))

9 . Find chess board corners

In [ ]:
# stop the iteration when specified accuracy, epsilon, is reached or
# specified number of iterations are completed.
criteria = (cv2.TERM_CRITERIA_EPS +
            cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# Vector for 3D points
threedpoints = []

# Vector for 2D points
twodpoints = []


# 3D points real world coordinates
objectp3d = np.zeros((1, CHECKERBOARD[0]
                    * CHECKERBOARD[1],
                    3), np.float32)
objectp3d[0, :, :2] = np.mgrid[0:CHECKERBOARD[0],
                            0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None


for filename in images:
    image = cv2.imread(filename)
    grayColor = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    # If desired number of corners are
    # found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(grayColor, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

    # If desired number of corners can be detected then,
    # refine the pixel coordinates and display
    # them on the images of checker board
    if ret:
        threedpoints.append(objectp3d)

        # Refining pixel coordinates
        # for given 2d points.
        corners2 = cv2.cornerSubPix(grayColor, corners, (11, 11), (-1, -1), criteria)

        twodpoints.append(corners2)

        # Draw and display the corners
        image = cv2.drawChessboardCorners(image, CHECKERBOARD, corners2, ret)
        cv2.imwrite(f"{save_path}/{filename}.jpg", image)
        # print(filename)
            
    else:
        pass
        # If for any reason cv can't find the chess board corners on an especific image, it will return the image name
        print('error: ', filename)

10 . Performe Calibration

In [ ]:
# Perform camera calibration by
# passing the value of above found out 3D points (threedpoints)
# and its corresponding pixel coordinates of the
# detected corners (twodpoints)
ret, matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(threedpoints, twodpoints, grayColor.shape[::-1], None, None)


# Displaying required output
print(" Camera matrix:")
print(matrix)

print("\n Distortion coefficient:")
print(distortion)

print("\n Rotation Vectors:")
print(r_vecs)

print("\n Translation Vectors:")
print(t_vecs)